This file is to run manually a head to head evaluation

In [ ]:
import copy
from agent_configs.cfr_config import CFRConfig
from active_player import ActivePlayer
from cfr_agent import CFRAgent
import torch
from cfr_network import CFRNetwork
from cfr_utils import evaluatebots, WrapperEnv, load_agents
import pyspiel
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
fhp = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 4, "numRanks":13, "numHoleCards": 2, "numBoardCards": "0 3", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
leduc = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 2, "numRanks":3, "numHoleCards": 1, "numBoardCards": "0 1", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
leducconfig = {"state_representation_size": 16}
fhpconfig = {"state_representation_size": 108}
leducgame = WrapperEnv(leduc)
fhpgame = WrapperEnv(fhp)

chosen_game = 'leduc'

hidden_dim = 128
input_dim = 16 if chosen_game == 'leduc' else 108
output_dim = 4
num_players = 2
replay_buffer_size = 4000000
minibatch_size = 10000
steps_per_epoch = 3000
traversals = 3000
training_steps = 20000
lr = 0.0001
optimizer = None

p_v_networks = {'input_shape':input_dim, 'output_shape':output_dim, 'hidden_size':hidden_dim, 'learning_rate':lr, 'optimizer':optimizer}
active_player_obj = ActivePlayer(num_players)
path1 = 'checkpoints/CFR_LEDUC_FULL/policy/linear/1033111/CFR_LEDUC_Full_50.pt'
path2 = 'checkpoints/CFR_LEDUC_FULL/policy/linear/2011916/CFR_LEDUC_Full_94.pt'
path3 = 'checkpoints/CFR_LEDUC_Full/policy/linear/3015760/CFR_LEDUC_Full_137.pt'
path4 = 'checkpoints/CFR_LEDUC_Full/policy/linear/4001364/CFR_LEDUC_Full_178.pt'
path5 = 'checkpoints/CFR_LEDUC_Full/policy/linear/5012235/CFR_LEDUC_Full_220.pt'
path6 = 'checkpoints/CFR_LEDUC_Full/policy/linear/6004659/CFR_LEDUC_Full_259.pt'
path7 = 'checkpoints/CFR_LEDUC_Full/policy/linear/7011009/CFR_LEDUC_Full_300.pt'
path8 = 'checkpoints/CFR_LEDUC_Full/policy/linear/8019516/CFR_LEDUC_Full_340.pt'
path9 = 'checkpoints/CFR_LEDUC_Full/policy/linear/9012089/CFR_LEDUC_Full_379.pt'
path10 = 'checkpoints/CFR_LEDUC_Full/policy/linear/10009409/CFR_LEDUC_Full_419.pt'
config = CFRConfig(
    config_dict={'network': {'policy': p_v_networks, 'value': p_v_networks, 'num_players':num_players},
                 'replay_buffer_size':replay_buffer_size,
                 'minibatch_size':minibatch_size,
                 'steps_per_epoch':steps_per_epoch,
                 'traversals': traversals,
                 'training_steps': training_steps,
                 'active_player_obj': active_player_obj,
                 },
    game_config={'num_players':num_players,
                 'observation_space':input_dim,
                 'action_space':4,},
)

agent1, agent2 = load_agents(path1, path2, p_v_networks, num_players)
agent3, agent4 = load_agents(path3, path4, p_v_networks, num_players)
agent5, agent6 = load_agents(path5, path6, p_v_networks, num_players)
agent7, agent8 = load_agents(path7, path8, p_v_networks, num_players)
agent9, agent10 = load_agents(path9, path10, p_v_networks, num_players)
agents = [agent1, agent2, agent3, agent4, agent5, agent6, agent7, agent8, agent9, agent10]

CFRConfig


In [ ]:
results = [[0 for p in range(len(agents))] for _ in range(len(agents))]
for i in range(len(agents)):
    for j in range(len(agents)):
        p1, p2 = evaluatebots(agent1, agent2, 10000, leducgame, config, input_dim)
        results[i][j] = np.mean(p1)/100

plt.figure(figsize=(10, 8))
plt.imshow(results, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title('Heatmap of Agent Performance')
plt.xlabel('Agent 2')
plt.ylabel('Agent 1')
plt.xticks(ticks=range(len(agents)), labels=[f'Agent {i+1}' for i in range(len(agents))])
plt.yticks(ticks=range(len(agents)), labels=[f'Agent {i+1}' for i in range(len(agents))])
plt.show()
# Save the plt to a file
plt.savefig('agent_performance_heatmap.png')
plt.close()

Player 1: -0.03725mbb/game
Player 2: 0.03725mbb/game
